# Programa general donde desarrollaremos la interfáz de usuario

**Requeriments**:  
nltk  
pillow  
request  
pandas  
beautifulsoup4  
textblob  
ipykernel  
scikit-learn

In [8]:
from queue import Empty
from tkinter import *
from tkinter import messagebox, ttk
from turtle import width
from PIL import ImageTk, Image
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
import requests as req
from bs4 import BeautifulSoup
# Imports de preprocesamiento de textos
import re
import requests
import string
import pickle
import pandas as pd
from textblob.en import subjectivity
# Calculamos la subjetividad y polaridad de cada frase
from textblob import TextBlob

bgColor = "#CFD9FF" # Azul claro
auxColor = "#FFD472" # Naranjita

# Modelos
modelNames = {"RFC" : "Random Forest Classifier",
        "LRC" : "Logistic Regression Classifier"}

# Creamos la ventana donde iremos colocando todos los elemenos necesarios
window = Tk()
window.title("TFM UEM. Daniel Riveros García 2022")
window.geometry("1070x500")
window.configure(bg=bgColor)

#--------------------------------- FUNCIONES DE PREPROCESAMIENTO ---------------------------------

stemmer = WordNetLemmatizer()
def lemmatization(text):
    
    text = text.split()
    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

# Auxiliar function for remove all emoji characters
# https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def removeEmoji(text):
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642"
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

# Preprocessor function
def preprocessor2(title_text):

    # Primero, segundo y cuarto, eliminiación de enlaces, menciones y hashtags - Incluimos eliminar espacios y caracteres raros
    title_text = title_text.apply(lambda x: re.sub(r'((https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b)|(MENTION[0-9]*)|(#[a-zA-Z0-9]*)', ' ', x, flags=re.MULTILINE))
    title_text = title_text.str.replace('\n',' ')
    title_text = title_text.str.replace('\t',' ')

    # Quitamos simbolos de puntuacion
    title_text = title_text.str.translate(str.maketrans('', '', string.punctuation))

    # Eliminamos los restos que no hayan sido eliminados y algunas expresiones que no aportan informacion
    title_text = title_text.apply(lambda x: re.sub(r'([^0-9a-zA-Z:,\s]+)|(rt)|(lol)|(lmao)|(lmfao)', '', x, flags=re.MULTILINE))

    # Despues de eliminar los simbolos de puntuacion, eliminamos los números
    title_text = title_text.apply(lambda x: re.sub(r'([0-9]+)', ' ', x, flags=re.MULTILINE))
    # Tercero, eliminiación de emojis
    title_text = title_text.apply(lambda x: removeEmoji(x))
    # Quitamos palabras sueltas con letras repetidas
    title_text = title_text.apply(lambda x: re.sub(r'(.)\1{3,}', ' ', x, flags=re.MULTILINE))

    # Ponemos todo en minusculas y aplicamos trim
    title_text = title_text.str.lower()
    title_text = title_text.str.strip()
    # Quitamos espacios sobrantes
    title_text = title_text.apply(lambda x: ' '.join([y for y in x.split(' ') if y != '']))

    # Lematizacion
    title_text = title_text.apply(lambda x: lemmatization(x))

    return title_text

#--------------------------------- CREACIÓN DE LAS FUNCIONES NECESARIAS ---------------------------------


# Boton para empezar el estudio de resultados
def startButtonF():

    # WebScrapping
    df = webScraping()

    # Si no ha habido error, continuamos
    if df is not None:
        # Aplicamos las transformaciones
        dfpre = preprocessor2(df['text'])

        # Lectura de modelos y vectorizer
        with open('Modelos/Usables/tfidfVectorizer', 'rb') as vect:
            vectorizer = pickle.load(vect)

        # Aplicamos el vectorizador
        X_internet = vectorizer.transform(dfpre)

        print(combo.get())
        # Depende del modelo activo, leemos uno u otro
        if combo.get() == modelNames.get('RFC'):
            print('rfc')
            with open('Modelos/Usables/bestRFC400Lem', 'rb') as trainded_model:
                model = pickle.load(trainded_model)
        elif combo.get() == modelNames.get('LRC'):
            print('lrc')
            with open('Modelos/Usables/bestLG400Lem', 'rb') as trainded_model:
                model = pickle.load(trainded_model)

        # Lista de machismo, por parrafo predice
        listSexism = model.predict(X_internet)

        count = 0
        for el in listSexism:
            if el == 1:
                count += 1

        percSexism = count/len(listSexism)

        # Analisis de Sentimientos
        pol, sub = sentimentAnalysis(df) # Pasamos el df original, mejor para el analisis de sentimientos
        updateLabels(str(dfpre.shape[0]), str(percSexism) + "%", str(round(pol, 6)), str(round(sub, 6)))

    else:
        updateLabels("", "", "", "")


def webScraping():
    # Obtenemos la url
    URL = urlImputLabel.get()
    urlOK = False
    if URL is not Empty and URL != '':
        page = requests.get(URL)
        urlOK = True
    
    # Si la url es correcta mostramos el titulo 
    if  urlOK and page.status_code == 200 and "www.independent.com" in URL:
        soup = BeautifulSoup(page.content, "html.parser")
        errLabel.place_forget()
        titleLabel.place(x=30, y=90)

        # Obtenemos el titulo de la noticia
        titleSoap = soup.find_all("h1", {"class": "entry-title"})
        for par in titleSoap:
            titleNew = par.getText()
            break

        titleTextLabel.config(text = titleNew)
        titleTextLabel.place(x=165, y=90)

        # Creamos el df con el contenido
        results = soup.find(id="pico")
        paragraphs = results.findAll('p')
        textos = []

        for par in paragraphs:
            textos.append(par.getText())

        df = pd.DataFrame(textos, columns=['text'])
        
    else: # Si no, el mensaje de error
        titleLabel.place_forget()
        titleTextLabel.place_forget()
        errLabel.place(x=30, y=90)
        df = None
    return df
    
def sentimentAnalysis(df):
    polarity = 0
    subjectivity = 0

    for index, row in df.iterrows():
        text = row['text']
        blob = TextBlob(text)

        localPol = 0
        localSub = 0
        for sentence in blob.sentences:
            localPol += sentence.sentiment.polarity
            localSub += sentence.sentiment.subjectivity 

        meanPol = localPol/len(blob.sentences)
        meanSub = localSub/len(blob.sentences)

        polarity += meanPol
        subjectivity += meanSub
    

    # Calculo de la media
    polarityMean = polarity/df.shape[0]
    subjectivityMean = subjectivity/df.shape[0]

    return polarityMean, subjectivityMean

# Funciones auxiliares del programa como tal
def updateLabels(numParr, percSexism, pol, sub):
    # Marcar los labels:

    numParrImputLabel['state'] = NORMAL
    numParrImputLabel.delete(0,END)
    numParrImputLabel.insert(0,numParr)
    numParrImputLabel['state'] = "readonly"

    sexismImputLabel['state'] = NORMAL
    sexismImputLabel.delete(0,END)
    sexismImputLabel.insert(0,percSexism)
    sexismImputLabel['state'] = "readonly"

    polImputLabel['state'] = NORMAL
    polImputLabel.delete(0,END)
    polImputLabel.insert(0,pol)
    polImputLabel['state'] = "readonly"

    subImputLabel['state'] = NORMAL
    subImputLabel.delete(0,END)
    subImputLabel.insert(0,sub)
    subImputLabel['state'] = "readonly"


#---------------------------------------

# Cuando seleccionemos un nuevo modelo, cambiaremos las imagenes
def selectionChange(event):
    print(combo.get())
    if combo.get() == modelNames.get('RFC'):
        lgnButton.place_forget()
        lgpButton.place_forget()
        rfcButton.place(x=xsel+30, y=ysel + 35)
        actModel = "RFC"

    elif combo.get() == modelNames.get('LRC'):
        rfcButton.place_forget()
        lgpButton.place(x=xsel+5, y=ysel + 35)
        lgnButton.place(x=xsel+160, y=ysel + 35)
        actModel = "LRC"



# Botones de información de las features -----

# Este boton será el de las features del modelo random forest
def openInfoRFC():

    # Habrá dos tipos de ventana, una para el modelo de regresion y otra para los arboles de decisión
    newWindow = Toplevel(window)
    newWindow.title("Información del Modelo")
    newWindow.configure(bg=bgColor)
    newWindow.geometry("850x650")

    titleLabel = Label(newWindow, text = 'Feature Importance - Random Forest Classifier', font=("Arial", 16), bg = auxColor)
    titleLabel.pack(side = TOP, fill= X)

    # Linea separadora (titulo - resto) ------------------
    separator = ttk.Separator(newWindow, orient='horizontal')
    separator.pack(fill='x')

    canvas = Canvas(newWindow, width = 100, height = 100)
    canvas.configure(bg=bgColor)
    canvas.pack(expand = YES, fill = BOTH)
    graph1 = PhotoImage(file = 'Imagenes/best_400_RFC_LEM_features.png')
    canvas.create_image(10, 5, image = graph1, anchor = NW)
    #assigned the gif1 to the canvas object
    canvas.graph1 = graph1

# Este boton será el de las features de LR Positivas
def openInfoLRPos():

    # Habrá dos tipos de ventana, una para el modelo de regresion y otra para los arboles de decisión
    newWindow = Toplevel(window)
    newWindow.title("Información del Modelo")
    newWindow.configure(bg=bgColor)
    newWindow.geometry("850x650")
   
    titleLabel = Label(newWindow, text = 'Feature Importance - Positive Class - Logistic Regression Classifier', font=("Arial", 16), bg = auxColor)
    titleLabel.pack(side = TOP, fill= X)
    # Linea separadora (titulo - resto) ------------------
    separator = ttk.Separator(newWindow, orient='horizontal')
    separator.pack(fill='x')

    canvas = Canvas(newWindow, width = 100, height = 100)
    canvas.configure(bg=bgColor)
    canvas.pack(expand = YES, fill = BOTH)
    graph1 = PhotoImage(file = 'Imagenes/best_400_LG_Pos_Lem_features.png')
    canvas.create_image(10, 5, image = graph1, anchor = NW)
    #assigned the gif1 to the canvas object
    canvas.graph1 = graph1
    
# Este boton será el de las features de LR Negativas
def openInfoLRNeg():

    # Habrá dos tipos de ventana, una para el modelo de regresion y otra para los arboles de decisión
    newWindow = Toplevel(window)
    newWindow.title("Información del Modelo")
    newWindow.configure(bg=bgColor)
    newWindow.geometry("850x650")
   
    titleLabel = Label(newWindow, text = 'Feature Importance - Negative Class - Logistic Regression Classifier', font=("Arial", 16), bg = auxColor)
    titleLabel.pack(side = TOP, fill= X)

     # Linea separadora (titulo - resto) ------------------
    separator = ttk.Separator(newWindow, orient='horizontal')
    separator.pack(fill='x')

    canvas = Canvas(newWindow, width = 100, height = 100)
    canvas.configure(bg=bgColor)
    canvas.pack(expand = YES, fill = BOTH)
    graph1 = PhotoImage(file = 'Imagenes/best_400_LG_Neg_Lem_features.png')
    canvas.create_image(10, 5, image = graph1, anchor = NW)
    #assigned the gif1 to the canvas object
    canvas.graph1 = graph1


#--------------------------------- CREACIÓN DE LOS ELEMENTOS ---------------------------------

# Título -----------------
titleLabel = Label(window, text = 'DETECCIÓN DE PATRONES MACHISTAS EN NOTICIAS DE INTERNET', font=("Arial", 23), bg = auxColor)
titleLabel.pack(side = TOP, fill= X)

# Linea separadora (titulo - resto) ------------------
separator = ttk.Separator(window, orient='horizontal')
separator.pack(fill='x')

# Apartado de entrada de url -----------------
urlLabel = Label(window, text = 'Inserte la url de la noticia a analizar:', font=("Arial", 12), bg = bgColor)
urlLabel.place(x=30, y=60)

urlImputLabel = Entry(window, font=("Arial", 12), width=60)
urlImputLabel.place(x=300, y=60)

startButton = Button(window, text='Comenzar', command= startButtonF, width=20)
startButton.place(x=875, y=58)

# Titulo de la noticia
titleLabel = Label(window, text = 'Título de la noticia:', font=("Arial bold", 12), bg = bgColor) 
#titleLabel.place(x=30, y=90)
titleTextLabel = Label(window, text = '.', font=("Arial bold", 12), bg = bgColor) 
#titleTextLabel.place(x=165, y=90)


# Mensaje de error
errLabel = Label(window, text = 'ERROR. La url especificada no es correcta.', font=("Arial bold", 14), bg = bgColor, fg= "red")
#errLabel.place(x=30, y=90)


# Selector de modelo ----------------

xsel = 100
ysel = 225

# Estilo
window.option_add('*TCombobox*Listbox.selectBackground', auxColor)
window.option_add('*TCombobox*Listbox.selectForeground', 'black')

comboLabel = Label(window, text = 'Seleccione el modelo que quiere usar*:', font=("Arial", 18), bg = bgColor)
comboLabel.place(x=xsel-60, y=ysel-50)

comboSubLabel = Label(window, text = '''*El modelo seleccionado solo cambiará los datos de machismo''', font=("Arial", 11), bg = bgColor)
comboSubLabel .place(x=xsel-60, y=ysel+70)

combo = ttk.Combobox(
    state="readonly",
    values=["Random Forest Classifier", "Logistic Regression Classifier"],
    width= 30,
    font=("Arial", 12)
)
combo.place(x=xsel, y=ysel)

# Dejamos preseleccionado el primero
combo.set("Random Forest Classifier")

# Aplicamos el bind para cuando cambiemosla selección
combo.bind("<<ComboboxSelected>>", selectionChange)

# Boton de más información con los datos del modelo ------------

# RFC
rfcButton = Button(window, text='Información Características', command= openInfoRFC, width=30)
# Por defecto vendrá activo el boton de RFC
rfcButton.place(x=xsel+30, y=ysel + 35)

# LR - Positive
lgpButton = Button(window, text='Inf. Pos. Caract.', command= openInfoLRPos, width=17)
#lgpButton.place(x=35, y=180)
# LR - Negative
lgnButton = Button(window, text='Inf. Neg. Caract.', command= openInfoLRNeg, width=17)
#lgnButton.place(x=190, y=180)

# Panel de resultados ------------

xResults = 600
yResults = 125

resultsLabel = Label(window, text = 'Resultados obtenidos:', font=("Arial", 20), bg = bgColor)
resultsLabel.place(x=625, y=yResults)

# Parrafos
numParrLabel = Label(window, text = 'Número de Párrafos:', font=("Arial", 14), bg = bgColor)
numParrLabel.place(x=xResults, y=yResults + 50)

numParrImputLabel = Entry(window, font=("Arial", 14), width=10, state="readonly")
numParrImputLabel.place(x=xResults+200, y=yResults + 50)

# Grado de sexismo

sexismLabel = Label(window, text = 'Porcentaje Machismo:', font=("Arial", 14), bg = bgColor)
sexismLabel.place(x=xResults, y=yResults+100)

sexismImputLabel = Entry(window, font=("Arial", 14), width=17, state="readonly")
sexismImputLabel.place(x=xResults+200, y=yResults+100)


# Grado de polaridad

pol1Label = Label(window, text = 'Grado de Polaridad:', font=("Arial", 14), bg = bgColor)
pol1Label.place(x=xResults, y=yResults+150)

pol2Label = Label(window, text = 'Donde -1 es muy negativa, 0 neutra y 1 muy positiva.', font=("Arial", 10), bg = bgColor)
pol2Label.place(x=xResults, y=yResults+175)

polImputLabel = Entry(window, font=("Arial", 14), width=17, state="readonly")
polImputLabel.place(x=xResults+200, y=yResults+150)

# Grado de subjetividad

sub1Label = Label(window, text = 'Grado de Subjetividad:', font=("Arial", 14), bg = bgColor)
sub1Label.place(x=xResults, y=yResults+200)

sub2Label = Label(window, text = 'Donde 0 es muy objetiva y 1 muy subjetiva.', font=("Arial", 10), bg = bgColor)
sub2Label.place(x=xResults, y=yResults+225)

subImputLabel = Entry(window, font=("Arial", 14), width=17, state="readonly")
subImputLabel.place(x=xResults+200, y=yResults+200)

# Boton de salir --------------

exitButton = Button(window, text='Salir', command= window.destroy, width=20)
exitButton.place(x=450, y=430)


# Pie de pagina --------------

# Título -----------------
feetLabel = Label(window, text = 'Universidad Europea de Madrid - Trabajo Fin Máster - Daniel Riveros García ©', font=("Arial", 12), bg = auxColor)
feetLabel.pack(side = BOTTOM, fill= X)

# Linea separadora (resto - pie de pagina) ------------------
separator2 = ttk.Separator(window, orient='horizontal')
separator2.pack(side = BOTTOM, fill='x')


# Main Loop de nuestra ventana, llevará el registro de todo lo que sucede en el interior de ella
window.mainloop()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Danie\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Danie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Danie\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
